In [37]:
import requests
import json
import pandas as pd
from google.cloud import storage
from google.cloud import bigquery
import sys
import time
import random
import datetime

In [25]:
#storage_client = storage.Client()
storage_client = storage.Client.from_service_account_json('../credentials/house-297220-545f62b34773.json')

In [26]:
# segments boundaries: split territory on the equal horizontal slices
bucket = storage_client.get_bucket('cities_boundaries')
bounds_blob = bucket.get_blob('usa/los_angeles.json')
bounds = json.loads(bounds_blob.download_as_string(client=None))

In [27]:
def make_top_url(bounds, request_id):
    part_1 = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"mapBounds":{'
    part_2 = '"west":{west},"east":{east},"south":{south},"north":{north}'.format(**bounds)
    part_3 = '},"isMapVisible":true,"filterState":{"isAllHomes":{"value":true}},"isListVisible":true,"mapZoom":9}&wants={"cat1":["mapResults"]}&requestId='
    
    return part_1 + part_2 + part_3 + str(request_id)


def make_raw_data(zpid):
    raw_data = {"operationName":"ForSaleDoubleScrollFullRenderQuery","variables":{"zpid":zpid,"contactFormRenderParameter":{"zpid":zpid,"platform":"desktop","isDoubleScroll":True}},"clientVersion":"home-details/6.0.11.3286.master.758d5aa","queryId":"2f76b63ef5b3d1d3398289fbdeee967c"}
    
    return raw_data


def make_low_url(zpid):
    low_url_template = 'https://www.zillow.com/graphql/?zpid={}&contactFormRenderParameter=&queryId=2f76b63ef5b3d1d3398289fbdeee967c&operationName=ForRentDoubleScrollFullRenderQuery'

    return low_url_template.format(zpid)

In [28]:
# headers
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]


headers_top = {
    'Host': 'www.zillow.com',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0'
}


headers_low = {
    'Host': 'www.zillow.com',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'content-type': 'text/plain',
    'Origin': 'https://www.zillow.com'
}

In [29]:
def transform_data(raw_details, zpid):
    # function transforms json details to data frame format
    
    # create dict to store the data to
    data = {'zpid': zpid, 'date_uploaded': datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f UTC')}
    
    # high level details
    high_level_data = raw_details.json()['data']['property']
    hig_level_props = ['city', 'homeStatus', 'price', 'streetAddress', 'datePosted', 'zipcode', 'latitude', 'longitude']
    for k in hig_level_props:
        if k in high_level_data.keys() and high_level_data[k] is not None:
            data[k] = high_level_data[k]

    # low level details
    low_level_data = raw_details.json()['data']['property']['resoFacts']
    low_level_props = [ 'bedrooms', 'bathrooms', 'bathroomsFull', 'bathroomsThreeQuarter', 'bathroomsHalf', 'bathroomsOneQuarter', 'bathroomsPartial', 'mainLevelBathrooms', 'basement', 'flooring', 'heating', 'hasHeating', 'cooling', 'hasCooling', 'appliances', 'laundryFeatures', 'fireplaces', 'fireplaceFeatures', 'hasFireplace', 'furnished', 'commonWalls', 'buildingArea', 'livingArea', 'aboveGradeFinishedArea', 'belowGradeFinishedArea', 'virtualTour', 'parking', 'parkingFeatures', 'garageSpaces', 'coveredSpaces', 'hasAttachedGarage', 'hasGarage', 'openParkingSpaces', 'hasOpenParking', 'carportSpaces', 'hasCarport', 'otherParking', 'accessibilityFeatures', 'levels', 'stories', 'entryLevel', 'entryLocation', 'hasPrivatePool', 'hasSpa', 'spaFeatures', 'exteriorFeatures', 'patioAndPorchFeatures', 'fencing', 'view', 'hasView', 'hasWaterfrontView', 'waterfrontFeatures', 'frontageType', 'frontageLength', 'topography', 'woodedArea', 'vegetation', 'canRaiseHorses', 'lotSize', 'lotSizeDimensions', 'otherStructures', 'additionalParcelsDescription', 'hasAdditionalParcels', 'parcelNumber', 'hasAttachedProperty', 'hasLandLease', 'landLeaseAmount', 'zoning', 'zoningDescription', 'homeType', 'architecturalStyle', 'constructionMaterials', 'foundationDetails', 'roofType', 'windowFeatures', 'propertyCondition', 'isNewConstruction', 'yearBuilt', 'developmentStatus', 'yearBuiltEffective', 'onMarketDate', 'builderModel', 'builderName', 'hasHomeWarranty', 'electric', 'hasElectricOnProperty', 'gas', 'sewer', 'waterSources', 'utilities', 'greenBuildingVerificationType', 'greenEnergyEfficient', 'greenIndoorAirQuality', 'greenSustainability', 'greenWaterConservation', 'numberOfUnitsInCommunity', 'numberOfUnitsVacant', 'storiesTotal', 'hasPetsAllowed', 'hasRentControl', 'buildingFeatures', 'structureType', 'buildingName', 'elementarySchool', 'elementarySchoolDistrict', 'middleOrJuniorSchool', 'middleOrJuniorSchoolDistrict', 'highSchool', 'highSchoolDistrict', 'securityFeatures', 'communityFeatures', 'isSeniorCommunity', 'cityRegion', 'associationFee', 'hasAssociation', 'associationAmenities', 'associationFeeIncludes', 'associationName', 'associationPhone', 'associationFee2', 'associationName2', 'associationPhone2', 'taxAssessedValue', 'taxAnnualAmount', 'listingId', 'buildingAreaSource']
    for k in low_level_props:
        if k in low_level_data.keys() and low_level_data[k] is not None and type(low_level_data[k]) != dict:
            value = low_level_data[k]
            if type(value) == list:
                value = ' | '.join(value)

            data[k] = value
    
    return data

In [30]:
def get_price_history(raw_details, zpid):
    if 'priceHistory' in raw_details.json()['data']['property'].keys():
        history = []
        for price_change in raw_details.json()['data']['property']['priceHistory']:
            history_date = datetime.datetime.fromtimestamp(int(price_change['time']/1000)).strftime('%Y-%m-%d %H:%M:%S.%f UTC')
            history_event = price_change['event']
            history_price = price_change['price']
            history_source = price_change['source']
            
            history.append({'zpid': zpid, 'date': history_date, 'event': history_event, 'price': history_price, 'source': history_source})
        
    else:
        history = False
    
    return history

In [31]:
query = """
SELECT zpid
  , price
FROM (
  SELECT zpid
    , price
    , date_uploaded
    , MAX(date_uploaded) OVER (PARTITION BY zpid) AS date_uploaded_last
  FROM `house-297220.usa.los_angeles_sale`
) AS a
WHERE date_uploaded = date_uploaded_last
"""

bigquery_client = bigquery.Client.from_service_account_json('../credentials/house-297220-545f62b34773.json')

query_job = bigquery_client.query(query)

existing_zpids_raw = query_job.result() 

existing_zpids = {}
for row in existing_zpids_raw:
    existing_zpids[str(row.zpid)] = row.price

In [ ]:
for segment in bounds.keys():
    # initialize metadata
    writen_objects = 0
    write_errors = 0
    total_results = 0
    broken_responses_top = ''
    broken_responses_low = []
    errors = []
    bq_errors = []
    
    
    url_top = make_top_url(bounds[segment], segment)
    headers_top['User-Agent'] = random.choice(user_agent_list)
    response_top = requests.get(url_top, headers=headers_top)

    if response_top.status_code != 200:
        time.sleep(60)
        headers_top['User-Agent'] = random.choice(user_agent_list)
        response_top = requests.get(url_top, headers=headers_top)

    if response_top.status_code != 200:
        broken_responses_top = str(response_top.status_code)

    else:
        if 'totalResultCount' in response_top.json()['cat1']['searchList'].keys():
            total_results = response_top.json()['cat1']['searchList']['totalResultCount']
        else:
            total_results = 'unknown'
        
        try:
            for result in response_top.json()['cat1']['searchResults']['mapResults']:
                # only if zpid and (if zpid doesn't exist yet or it had different price)
                if 'zpid' in result.keys() and (result['zpid'] not in existing_zpids.keys() or existing_zpids[result['zpid']] != int(result['hdpData']['homeInfo']['price'])):
                    zpid = result['zpid']


                    url_low = make_low_url(zpid)
                    raw_data = make_raw_data(zpid)
                    headers_low['User-Agent'] = random.choice(user_agent_list)
                    raw_details = requests.post(url_low, headers=headers_low, json=raw_data)

                    if raw_details.status_code != 200:
                        time.sleep(60)
                        headers_low['User-Agent'] = random.choice(user_agent_list)
                        raw_details = requests.post(url_low, headers=headers_low, json=raw_data)

                    if raw_details.status_code != 200:
                        broken_responses_low.append(str(raw_details.status_code))

                    else:
                        data = transform_data(raw_details, zpid)                          
                        bq_error = bigquery_client.insert_rows_json('house-297220.usa.los_angeles_sale', [data,])
                        if bq_error == []:
                            writen_objects += 1
                        else:
                            write_errors += 1
                            bq_errors.append(str(bq_error))
                        
                        
                        price_history = get_price_history(raw_details, zpid)
                        if price_history != False:
                            e = bigquery_client.insert_rows_json('house-297220.usa.los_angeles_sale_price_history', price_history)

                    time.sleep(3)
        
        except:
            ee = str(sys.exc_info()[0])
            errors.append(ee)
            print(ee)
            time.sleep(3)
            continue
            

        
        
        metadata = {
            'type': 'sale',
            'city': 'Los Angeles',
            'segment': segment,
            'date': datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f UTC'),
            'total_results': total_results,
            'writen_objects': writen_objects,
            'write_errors': write_errors,
            'broken_responses_top': broken_responses_top,
            'broken_responses_low': ' | '.join(broken_responses_low),
            'errors': ' | '.join(errors),
            'bq_errors': ' | '.join(bq_errors)
        } 
        
        print(metadata)
        
        try:
            e = bigquery_client.insert_rows_json('house-297220.usa.metadata', [metadata,])
            
        except:
            continue   

{'type': 'sale', 'city': 'New York', 'segment': '1', 'date': '2020-12-14 21:29:24.570398 UTC', 'total_results': 42, 'writen_objects': 39, 'write_errors': 0, 'broken_responses_top': '', 'broken_responses_low': '', 'errors': '', 'bq_errors': ''}
{'type': 'sale', 'city': 'New York', 'segment': '2', 'date': '2020-12-14 21:31:56.052939 UTC', 'total_results': 38, 'writen_objects': 35, 'write_errors': 0, 'broken_responses_top': '', 'broken_responses_low': '', 'errors': '', 'bq_errors': ''}
<class 'google.api_core.exceptions.BadRequest'>
